In [1]:
import pandas as pd
import numpy as np
from PIL import Image, ImageFont, ImageDraw

In [2]:
data = pd.read_csv('20191130 NCST K vs NCUN 31 PLAYS K.KR.P.PR.FG.FGB.csv')

In [3]:
team = "NCST"
# ALL PLAYS DATAFRAMES

p = data[(data["pff_SPECIALTEAMSTYPE"] == "PUNT") & (data["pff_OFFTEAM"] == team)]
kor = data[(data["pff_SPECIALTEAMSTYPE"] == "KICKOFF") & (data["pff_OFFTEAM"] == team)]

pr = data[(data["pff_SPECIALTEAMSTYPE"] == "PUNT") & (data["pff_DEFTEAM"] == team)]
ko = data[(data["pff_SPECIALTEAMSTYPE"] == "KICKOFF") & (data["pff_DEFTEAM"] == team)]

epfg = data[(data["pff_SPECIALTEAMSTYPE"] == "EXTRA POINT") | (data["pff_SPECIALTEAMSTYPE"] == "FIELD GOAL")]
fg = epfg[epfg["pff_OFFTEAM"] == team]
fgb = epfg[epfg["pff_DEFTEAM"] == team]

In [24]:
p["pff_OFFPLAYERSRATINGS"]

11    NCST D11 (0.0); NCST D13 (0.0); NCST D21 (0.0)...
12    NCST D11 (0.0); NCST D19 (+0.5); NCST D21 (0.0...
13    NCST D11 (0.0); NCST D13 (0.0); NCST D21 (0.0)...
14    NCST D11 (0.0); NCST D19 (0.0); NCST D21 (0.0)...
15    NCST D11 (0.0); NCST D13 (0.0); NCST D21 (0.0)...
16    NCST D11 (0.0); NCST D13 (0.0); NCST D21 (0.0)...
Name: pff_OFFPLAYERSRATINGS, dtype: object

In [25]:
df = pd.DataFrame(columns = ['POSITION', 'PLAYER', 'COUNT', 'RATING'])

formations = ko["pff_DEFPLAYERS"]
ratings = ko["pff_DEFPLAYERSRATINGS"]

for i in range(len(formations)):
    formation = formations[i]
    rating = ratings[i]
    
    formation = formation.replace('NCST ', '').replace('(', '').replace(')', '')
    rating = rating.replace('NCST ', '').replace('(', '').replace(')', '')
    
    player_positions = formation.split('; ')
    player_ratings = rating.split('; ')
    
    for j in range(len(player_positions)):
        player_position = player_positions[j]
        player_rating = player_ratings[j]
        
        player, position = player_position.split(' ')
        player, rating = player_rating.split(' ')
        
        new_player = {"POSITION": position, "PLAYER": player, "COUNT": int(1), "RATING": float(rating)}
        df = df.append(new_player, ignore_index = True)
        
df["COUNT"] = df["COUNT"].astype('int')
playerPositionCountsRatings = df.groupby(['POSITION', 'PLAYER'], as_index=False)['COUNT', 'RATING'].sum()

/Users/himol7/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [26]:
playerPositionCountsRatings

,POSITION,PLAYER,COUNT,RATING
0,L1,34,1,0.0
1,L1,D21,1,0.0
2,L1,D25,1,0.0
3,L2,38,2,0.0
4,L2,D19,1,0.5
5,L3,37,2,0.0
6,L3,D30,1,-0.5
7,L4,36,1,0.0
8,L4,D11,1,1.0
9,L4,D22,1,0.0


In [ ]:
print(players)
print(rating)

In [4]:
plays = ko["pff_DEFPLAYERS"]

In [9]:
ko["pff_DEFPLAYERSRATINGS"]

0    NCST D11 (+0.5); NCST D19 (0.0); NCST D21 (0.0...
1    NCST D11 (+1.0); NCST D19 (+0.5); NCST D21 (0....
2    NCST D11 (0.0); NCST D19 (0.0); NCST D21 (0.0)...
Name: pff_DEFPLAYERSRATINGS, dtype: object

In [6]:
print(plays)

0    NCST D11 (R4); NCST D19 (R2); NCST D21 (R1); N...
1    NCST D11 (L4); NCST D19 (L2); NCST D21 (L1); N...
2    NCST D11 (R4); NCST D19 (R2); NCST D21 (R1); N...
Name: pff_DEFPLAYERS, dtype: object


In [5]:
df = pd.DataFrame(columns = ['POSITION', 'PLAYER', 'COUNT'])
for play in plays:
    play = play.replace('NCST ', '').replace('(', '').replace(')', '')
    for player in play.split('; '):
        player_no, player_position = player.split(' ')
        new_player = {"POSITION": player_position, "PLAYER": player_no, "COUNT": 1}
        df = df.append(new_player, ignore_index = True)
        position_player_total = df.groupby(['POSITION', 'PLAYER'], as_index=False).sum()

In [6]:
coordinates = {
    "R1" : (20, 140),
    "R2" : (65, 140),
    "R3" : (110, 140),
    "R4" : (155, 140),
    "R5" : (200, 140),
    "PK" : (245, 140),
    "L5" : (290, 140),
    "L4" : (335, 140),
    "L3" : (380, 140),
    "L2" : (425, 140),
    "L1" : (470, 140)
}

In [8]:
image = Image.open('kickoff_coverage.png')
font = ImageFont.truetype('arial_bold.ttf', size=10)
draw = ImageDraw.Draw(image)

for position, positional_group in position_player_total.groupby(['POSITION']):
    (x, y) = coordinates.get(position)
    message = ''
    for index, player in positional_group.iterrows():
        message = message + str(player["PLAYER"]) + " " + str(player["COUNT"]) + '\n'
    color = 'rgb(0, 0, 0)'
    draw.text((x, y), message, fill=color, font=font)
image.save('trial2.png')